Importing the necessities


In [3]:
#importing the lib
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 

Loading the dataset 


In [4]:
import chardet
with open("spam.csv", 'rb') as f:
    result = chardet.detect(f.read())
df = pd.read_csv("spam.csv", encoding=result['encoding'])


In [5]:
#overviewing the dataset 
df.sample(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
3100,ham,Pathaya enketa maraikara pa',NaN,NaN,NaN
825,ham,Hmm .. Bits and pieces lol ... *sighs* ...,NaN,NaN,NaN
869,spam,What do U want for Xmas? How about 100 free te...,NaN,NaN,NaN
2680,ham,Solve d Case : A Man Was Found Murdered On &l...,NaN,NaN,NaN
2914,ham,Nothing lor... A bit bored too... Then y dun u...,NaN,NaN,NaN


Data Cleaning 

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


From above , we can clearly see in columns 2,3 and 4 most of the values are null . So , dropping them is the best.

In [7]:
#dropping last three columns 
df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace=True)

In [8]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
#renaming the columns , since there are not informative 
df.rename(columns={'v1':'target','v2':'text'},inplace=True)
df.sample(5)

,target,text
4253,ham,"How about clothes, jewelry, and trips?"
4894,spam,Want the latest Video handset? 750 anytime any...
3253,ham,I can make lasagna for you... vodka...
1351,ham,Yo theres no class tmrw right?
4968,ham,I will cme i want to go to hos 2morow. After t...


In [10]:
#label encoding 
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df.target=encoder.fit_transform(df['target'])
df.head()

,target,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [11]:
#missing values 
df.isnull().sum()

target    0
text      0
dtype: int64

In [13]:
#duplicate values 
df.duplicated().sum()

np.int64(403)

In [15]:
df.shape


(5572, 2)

In [17]:
df.drop_duplicates(keep='first',inplace=True)
df.shape

(5169, 2)